# Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto

# I. Create the dataframe

### Import useful packages for 1st part of assignment

In [1]:
import numpy as np
import pandas as pd
import requests

### 1. Obtain data from wiki page and transform it into a pandas frame

In [2]:
# EXTRACT TABLE FROM WIKI WEBPAGE
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

r = requests.get(url)
df_list = pd.read_html(r.text) # this parses all the tables in webpages to a list
df = df_list[0]

df.head()


,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


### 2. Only process the cells that have an assigned borough. 

In [3]:
#Ignore cells with a borough that is Not assigned.
df_filtered = df[(df.Borough != 'Not assigned')]
#Sort rows for easier comparison
df_filtered.sort_values(by=['Postal Code'])

,Postal Code,Borough,Neighborhood
9,M1B,Scarborough,"Malvern, Rouge"
18,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
27,M1E,Scarborough,"Guildwood, Morningside, West Hill"
36,M1G,Scarborough,Woburn
45,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
107,M9P,Etobicoke,Westmount
116,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ..."
143,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."


### 3. Combine rows with the same Postal Code

In [4]:
#Groupby and concatenate neighborhoods 
grp = df_filtered.groupby('Postal Code').agg(Neighborhood=('Neighborhood', ', '.join)).reset_index()

# Add Borough column back
grp['Borough'] = pd.Series([df_filtered.loc[df_filtered['Postal Code']==x, 'Borough'].iloc[0] 
                     for x in grp['Postal Code']])
# Sort rows for easier comparison
grp.sort_values(by=['Postal Code'], inplace=True)
grp
#Did not have any effect, as the wiki page table was already updated to show the information in this way.

,Postal Code,Neighborhood,Borough
0,M1B,"Malvern, Rouge",Scarborough
1,M1C,"Rouge Hill, Port Union, Highland Creek",Scarborough
2,M1E,"Guildwood, Morningside, West Hill",Scarborough
3,M1G,Woburn,Scarborough
4,M1H,Cedarbrae,Scarborough
...,...,...,...
98,M9N,Weston,York
99,M9P,Westmount,Etobicoke
100,M9R,"Kingsview Village, St. Phillips, Martin Grove ...",Etobicoke
101,M9V,"South Steeles, Silverstone, Humbergate, Jamest...",Etobicoke


### 4. If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [5]:
#Create a mask
m = ~(grp == 'Not Assigned')

#use mask to assign value of Borough to neighborhood
grp1 = grp.where(m, other=grp['Borough'], axis=0)
grp1


,Postal Code,Neighborhood,Borough
0,M1B,"Malvern, Rouge",Scarborough
1,M1C,"Rouge Hill, Port Union, Highland Creek",Scarborough
2,M1E,"Guildwood, Morningside, West Hill",Scarborough
3,M1G,Woburn,Scarborough
4,M1H,Cedarbrae,Scarborough
...,...,...,...
98,M9N,Weston,York
99,M9P,Westmount,Etobicoke
100,M9R,"Kingsview Village, St. Phillips, Martin Grove ...",Etobicoke
101,M9V,"South Steeles, Silverstone, Humbergate, Jamest...",Etobicoke


In [6]:
#Let's verify if we have any "Not Assigned" values after this process
NA_count=grp1['Neighborhood'].str.contains('Assigned').sum()
if NA_count>0:
    print ("There are {m} Not Assigned".format(m=NA_count))

In [7]:
#Let's verify if we had any "Not Assigned" values before this process
NA_count=grp['Neighborhood'].str.contains('Assigned').sum()
if NA_count>0:
    print ("There are {m} Not Assigned".format(m=NA_count))

# It seems we were ok to start with.

### 5. Let's see the shape of our dataframe

In [8]:
grp1.shape

(103, 3)

# II. Get the latitude and the longitude coordinates of each neighborhood

### Install Opencage
I tried initially using geocoder, but it did not work, it din not repsond at all.

#### Decided to use another API for this task: Opencage

In [9]:
#Install Opencage
pip install opencage

SyntaxError: invalid syntax (<ipython-input-9-8e377176b3a7>, line 2)

In [10]:
#import geocoder
from opencage.geocoder import OpenCageGeocode

In [11]:
#Setup with my API key
key = '9fe4636912da41de985879968693ca55'
geocoder = OpenCageGeocode(key)


In [12]:
#test the way it works. Decided on the querry format: "City, Postal Code" in order to gen latitude and longitude
results = geocoder.geocode('Toronto, M9N')
lat = results[0]['geometry']['lat']
lng = results[0]['geometry']['lng']
print(lat)
print(lng)

43.7068
-79.517


### Use opencage to get lists of latitude and longitudes

In [13]:
# create empty lists
list_lat = []

list_long = []

for index, row in grp1.iterrows(): # iterate over rows in dataframe
    Postal = row['Postal Code']
    City = 'Toronto'       
    query = str(City)+', '+str(Postal)

    results = geocoder.geocode(query)   
    lat = results[0]['geometry']['lat']
    long = results[0]['geometry']['lng']

    list_lat.append(lat)
    list_long.append(long)


### Add the two lists as columns in my data frame

In [14]:
grp1['Latitude'] = list_lat   
grp1['Longitude'] = list_long
grp1

,Postal Code,Neighborhood,Borough,Latitude,Longitude
0,M1B,"Malvern, Rouge",Scarborough,-33.013441,151.594204
1,M1C,"Rouge Hill, Port Union, Highland Creek",Scarborough,-33.013441,151.594204
2,M1E,"Guildwood, Morningside, West Hill",Scarborough,43.767800,-79.186600
3,M1G,Woburn,Scarborough,43.765717,-79.221898
4,M1H,Cedarbrae,Scarborough,43.768600,-79.238900
...,...,...,...,...,...
98,M9N,Weston,York,43.706800,-79.517000
99,M9P,Westmount,Etobicoke,43.694900,-79.532300
100,M9R,"Kingsview Village, St. Phillips, Martin Grove ...",Etobicoke,43.695166,-79.550890
101,M9V,"South Steeles, Silverstone, Humbergate, Jamest...",Etobicoke,43.743200,-79.587600


In [88]:
#check if all lines have latitude and longitude
print(grp1['Latitude'].isnull().values.any())
print(grp1['Longitude'].isnull().values.any())

False
False


# III. Explore and cluster the neighborhoods in Toronto

### 1. Installing useful packages

In [15]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library


Solving environment: ...working... done

# All requested packages already installed.



In [16]:
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\Claudia\anaconda3

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.22.0               |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          97 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-1.22.0-pyh9f0ad1d_0




geopy-1.22.0         | 63 KB     |            |   0% 
geopy-1.22.0         | 63 KB     | ##5        |  26% 
geopy-1.22.0         | 63 KB     | ########## | 100% 

geographiclib-1.50   | 34 KB     |            |   0% 
geographiclib-1.5

### 2. Get Toronto coordinates in order to create map

In [17]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 43.6534817, -79.3839347.


### 3. Create map of Toronto and add markers for each neighborhood in our dataframe

In [21]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(grp1['Latitude'], grp1['Longitude'], grp1['Borough'], grp1['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### 4. Get data from Foursquare on venues in each neighborhood

#### 4.1.Define Foursquare credentials and version

In [24]:
CLIENT_ID = 'NZPX4DYNBR5CAEDUGALEG5B4QVUNIDBSBPZZ1PD10UL1NPTJ' 
CLIENT_SECRET = 'V3RCYBQOP5HHGJOXCJHOK5WNZUKOVBKNFKYREILBGEXHDASV' 
VERSION = '20180604'


#### 4.2. Define a function that gets nearby venues with name, location and category

In [27]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#Limit the number of results by 100
LIMIT = 100

#### 4.3. Run the above function on each neighborhood in Toronto and create a new dataframe with venues in Toronto

In [116]:
#run analysis on a subset of neighborhoods from boroughs which contain the name Toronto - 39 total rows
toronto_subdata = grp1[grp1['Borough'].str.contains("Toronto")]

toronto_subdata.head()

,Postal Code,Neighborhood,Borough,Latitude,Longitude
37,M4E,The Beaches,East Toronto,43.67840,-79.294100
41,M4K,"The Danforth West, Riverdale",East Toronto,43.68030,-79.353800
42,M4L,"India Bazaar, The Beaches West",East Toronto,43.67276,-79.304058
43,M4M,Studio District,East Toronto,43.65610,-79.340600
44,M4N,Lawrence Park,Central Toronto,43.73010,-79.393500


In [117]:
toronto_venues = getNearbyVenues(names=toronto_subdata['Neighborhood'],
                                   latitudes=toronto_subdata['Latitude'],
                                   longitudes=toronto_subdata['Longitude']
                                  )

The Beaches
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West, Lawrence Park
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North & West, Forest Hill Road Park
The Annex, North Midtown, Yorkville
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockton, Parkdale Village, Exhibition Place
High 

In [118]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.6784,-79.2941,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.6784,-79.2941,Beaches Bake Shop,43.680363,-79.289692,Bakery
2,The Beaches,43.6784,-79.2941,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
3,The Beaches,43.6784,-79.2941,The Beech Tree,43.680493,-79.288846,Gastropub
4,The Beaches,43.6784,-79.2941,Grover Pub and Grub,43.679181,-79.297215,Pub


#### 4.4. Analyze each neighborhood

In [121]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

#There seemns to be a neighborhood column in dataframe, so i'll drop it (it has values of 0 or 1)
toronto_onehot= toronto_onehot.drop('Neighborhood', axis=1)

#insert Neighborhood column from venues dataframe back
toronto_onehot.insert(0, 'Neighborhood', toronto_venues['Neighborhood'], True) 


toronto_onehot.head()

,Neighborhood,Afghan Restaurant,American Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [122]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()



,Neighborhood,Afghan Restaurant,American Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Berczy Park,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000
2,"Business reply mail Processing Centre, South C...",0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.071429
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.0,0.0,0.0,0.011364,0.000000,0.011364,0.0,0.0,0.0,...,0.0,0.011364,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.022727
4,Central Bay Street,0.0,0.0,0.0,0.000000,0.016129,0.000000,0.0,0.0,0.0,...,0.0,0.000000,0.016129,0.016129,0.0,0.0,0.016129,0.0,0.0,0.000000


In [123]:
toronto_grouped.shape

(39, 211)

#### 4.5. Create a dataframe which contains top 10 venues for each neighborhood 

In [124]:
#define function which sorts venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [125]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Pet Store,Home Service,Lawyer,Doner Restaurant,Fish Market,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Event Space,Ethiopian Restaurant
1,"Brockton, Parkdale Village, Exhibition Place",Pet Store,Home Service,Lawyer,Doner Restaurant,Fish Market,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Event Space,Ethiopian Restaurant
2,"Business reply mail Processing Centre, South C...",Italian Restaurant,Coffee Shop,Restaurant,Bank,Martial Arts Dojo,Breakfast Spot,Sushi Restaurant,Bookstore,Japanese Restaurant,Yoga Studio
3,"CN Tower, King and Spadina, Railway Lands, Har...",Coffee Shop,Hotel,Café,Pizza Place,Gym,Restaurant,Italian Restaurant,Yoga Studio,Theater,Pub
4,Central Bay Street,Coffee Shop,Middle Eastern Restaurant,Sandwich Place,Café,Bubble Tea Shop,Italian Restaurant,Japanese Restaurant,Gastropub,Breakfast Spot,Shopping Mall


### 5. Cluster neighborhoods

#### 5.1. Run k-means to cluster neighborhoods

In [140]:
# set the parameters for clustering
k_means = KMeans(init="k-means++", n_clusters=5, n_init=12)

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans.fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:20] 

array([3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 0, 4, 3, 3, 4, 4, 1, 4])

#### 5.2. Create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood

In [141]:
# add clustering labels
neighborhoods_venues_sorted.drop('Cluster Labels', axis=1, inplace=True)
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_m = toronto_subdata

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_m = toronto_m.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_m.head()

,Postal Code,Neighborhood,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,M4E,The Beaches,East Toronto,43.67840,-79.294100,4,Pub,Health Food Store,Gastropub,Bakery,Cheese Shop,Trail,Yoga Studio,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant
41,M4K,"The Danforth West, Riverdale",East Toronto,43.68030,-79.353800,4,Greek Restaurant,Ice Cream Shop,Italian Restaurant,Café,Grocery Store,Bubble Tea Shop,Spa,Cocktail Bar,Restaurant,Coffee Shop
42,M4L,"India Bazaar, The Beaches West",East Toronto,43.67276,-79.304058,4,Pub,BBQ Joint,Bakery,Coffee Shop,Food,Bookstore,Japanese Restaurant,Breakfast Spot,Sporting Goods Shop,Middle Eastern Restaurant
43,M4M,Studio District,East Toronto,43.65610,-79.340600,4,Baseball Field,Park,Coffee Shop,Diner,Garden Center,Gym,Performing Arts Venue,Coworking Space,Filipino Restaurant,Fast Food Restaurant
44,M4N,Lawrence Park,Central Toronto,43.73010,-79.393500,1,Photography Studio,Park,Doner Restaurant,Fish Market,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Event Space,Ethiopian Restaurant,Electronics Store


#### 5.3. Create map to visualize clusters

In [142]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_m['Latitude'], toronto_m['Longitude'], toronto_m['Neighborhood'], toronto_m['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### 5.4. Explore clusters

#### 1st cluster

In [143]:
toronto_m.loc[toronto_m['Cluster Labels'] == 0, toronto_m.columns[[1] + list(range(5, toronto_m.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
46,"North Toronto West, Lawrence Park",0,Gym Pool,Playground,Park,Garden,Yoga Studio,Doner Restaurant,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Event Space
48,"Moore Park, Summerhill East",0,Thai Restaurant,Tennis Court,Gym,Grocery Store,Park,Playground,Eastern European Restaurant,Doner Restaurant,Donut Shop,Dumpling Restaurant
50,Rosedale,0,Playground,Grocery Store,Park,Candy Store,Yoga Studio,Doner Restaurant,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Event Space
64,"Forest Hill North & West, Forest Hill Road Park",0,Trail,Bus Line,Park,Yoga Studio,Donut Shop,Fish Market,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Event Space


#### 2nd cluster

In [144]:
toronto_m.loc[toronto_m['Cluster Labels'] == 1, toronto_m.columns[[1] + list(range(5, toronto_m.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
44,Lawrence Park,1,Photography Studio,Park,Doner Restaurant,Fish Market,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Event Space,Ethiopian Restaurant,Electronics Store


#### 3rd cluster

In [145]:
toronto_m.loc[toronto_m['Cluster Labels'] == 2, toronto_m.columns[[1] + list(range(5, toronto_m.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
63,Roselawn,2,Clothing Store,Yoga Studio,Food & Drink Shop,Flower Shop,Fish Market,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Event Space,Ethiopian Restaurant


#### 4th cluster

In [146]:
toronto_m.loc[toronto_m['Cluster Labels'] == 3, toronto_m.columns[[1] + list(range(5, toronto_m.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
56,Berczy Park,3,Pet Store,Home Service,Lawyer,Doner Restaurant,Fish Market,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Event Space,Ethiopian Restaurant
58,"Richmond, Adelaide, King",3,Pet Store,Home Service,Lawyer,Doner Restaurant,Fish Market,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Event Space,Ethiopian Restaurant
59,"Harbourfront East, Union Station, Toronto Islands",3,Pet Store,Home Service,Lawyer,Doner Restaurant,Fish Market,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Event Space,Ethiopian Restaurant
78,"Brockton, Parkdale Village, Exhibition Place",3,Pet Store,Home Service,Lawyer,Doner Restaurant,Fish Market,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Event Space,Ethiopian Restaurant
82,"High Park, The Junction South",3,Pet Store,Home Service,Lawyer,Doner Restaurant,Fish Market,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Event Space,Ethiopian Restaurant
84,"Runnymede, Swansea",3,Pet Store,Home Service,Lawyer,Doner Restaurant,Fish Market,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Event Space,Ethiopian Restaurant


#### 5th cluster

In [147]:
toronto_m.loc[toronto_m['Cluster Labels'] == 4, toronto_m.columns[[1] + list(range(5, toronto_m.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,The Beaches,4,Pub,Health Food Store,Gastropub,Bakery,Cheese Shop,Trail,Yoga Studio,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant
41,"The Danforth West, Riverdale",4,Greek Restaurant,Ice Cream Shop,Italian Restaurant,Café,Grocery Store,Bubble Tea Shop,Spa,Cocktail Bar,Restaurant,Coffee Shop
42,"India Bazaar, The Beaches West",4,Pub,BBQ Joint,Bakery,Coffee Shop,Food,Bookstore,Japanese Restaurant,Breakfast Spot,Sporting Goods Shop,Middle Eastern Restaurant
43,Studio District,4,Baseball Field,Park,Coffee Shop,Diner,Garden Center,Gym,Performing Arts Venue,Coworking Space,Filipino Restaurant,Fast Food Restaurant
45,Davisville North,4,Food & Drink Shop,Gym,Park,Department Store,Breakfast Spot,Dog Run,Dumpling Restaurant,Flower Shop,Fish Market,Filipino Restaurant
47,Davisville,4,Café,Sandwich Place,Italian Restaurant,Coffee Shop,Dessert Shop,Indian Restaurant,Diner,Seafood Restaurant,Restaurant,Sushi Restaurant
49,"Summerhill West, Rathnelly, South Hill, Forest...",4,Light Rail Station,Coffee Shop,Liquor Store,Supermarket,Athletics & Sports,Dumpling Restaurant,Flower Shop,Fish Market,Filipino Restaurant,Fast Food Restaurant
51,"St. James Town, Cabbagetown",4,Coffee Shop,Pizza Place,Restaurant,Café,Chinese Restaurant,Bakery,Pub,Italian Restaurant,Pet Store,Convenience Store
52,Church and Wellesley,4,Sushi Restaurant,Coffee Shop,Japanese Restaurant,Gay Bar,Restaurant,Yoga Studio,Bubble Tea Shop,Burger Joint,Grocery Store,Hotel
53,"Regent Park, Harbourfront",4,Coffee Shop,Breakfast Spot,Restaurant,Yoga Studio,Theater,Spa,Event Space,Food Truck,Electronics Store,Beer Store
